# Rekognitionを利用した猫判定の実施
## 目次
1. 事前準備
    * S3バケットの指定
2. 学習の入力となる画像イメージのアップロード
3. 【AWSコンソール操作（Rekognition）】　プロジェクト作成
4. 【AWSコンソール操作（Rekognition）】　データセット　
5. 【AWSコンソール操作（Rekognition）】　ラベリング　　
6. 【AWSコンソール操作（Rekognition）】　modelのトレーニング開始　
7. modelの起動
8. modelを利用した画像判定
9. modelの停止

## 1. 事前準備
<ユーザーID>の内容を自分のユーザーIDへ置き換えて下さい
* 例）S3_BUCKET='training-user1-s3'

In [ ]:
S3_BUCKET='training-<ユーザーID>-s3'

## 2. 学習の入力となる画像イメージのアップロード

In [ ]:
IMAGES_PATH='training1/images'
!aws s3 cp ./train_raw_images s3://{S3_BUCKET}/{IMAGES_PATH} --recursive

> #### 【チェックポイント】S3へアップロードされていることを確認
> ![](./images_for_ipynb/t1-1.png)

## 3. 【AWSコンソール操作（Rekognition）】　プロジェクト作成
* Rekognitionサービスの選択
    * 検索へ`rekognition`と入力すると候補が表示されますのでサービスを選択してください

![](./images_for_ipynb/t1-2.png)
* 「カスタムラベルを利用」を選択し、カスタムラベルの設定を行っていく

![](./images_for_ipynb/t1-3.png)
* Projectを新規作成していきます

![](./images_for_ipynb/t1-4.png)
* Project 名を指定し新規プロジェクトを作成します。
    * `Project Name`は <ユーザーID>-cats-project を設定してください

![](./images_for_ipynb/t1-5.png)

## 4. 【AWSコンソール操作（Rekognition）】　データセット　
* 作成したProjectにて`Create Dataset`を押下しデータセットを設定していきます

![](./images_for_ipynb/t1-6.png)
* データセットの設定を行います
    * `Start with a single Dataset` を選択
    * `Import images from S3 buket`を選択
    * `S3 URI`へアップロードしたS3のURIを設定する
    * `Create Dataset` を押下しデータセットの作成を行う

![](./images_for_ipynb/t1-7.png)


## 5. 【AWSコンソール操作（Rekognition）】　ラベリング

* データセットを選択しラベリングを開始していきます
    * `Start labeling` を押下します

![](./images_for_ipynb/t1-dataset-1.png)

* ラベルの選択肢を作っていきます


![](./images_for_ipynb/t1-dataset-2.png)

* tamaとmikeが下記のとおりとなるように設定しSaveします


![](./images_for_ipynb/t1-dataset-3.png)

* 画像を選択します


![](./images_for_ipynb/t1-dataset-4.png)

* 選択した画像がどちらなのか選択します


![](./images_for_ipynb/t1-dataset-5.png)

* ラベルが設定されていることを確認します（これをすべての画像で実施します）


![](./images_for_ipynb/t1-dataset-6.png)

* 全てのラベリングが終了したら変更内容を保存します


![](./images_for_ipynb/t1-dataset-7.png)



## 6. 【AWSコンソール操作（Rekognition）】　modelのトレーニング開始
* ラベリングの結果を確認しmodelのトレーニング開始を開始します
   * LabeledがImagesと同じ数になっていることを確認（すべてラベリングされていることを確認）
   * `Train model` を押下


![](./images_for_ipynb/t1-model-1.png)

* 内容はそのままで`Train model` を押下し、modelのトレーニング開始を開始します
    * `Do you want to train your model?`とダイアログで聞かれますが`Train model` を押下する
    * modelのトレーニングはとても時間がかかります（40分程度）


![](./images_for_ipynb/t1-model-2.png)


## 7. modelの起動
![](./images_for_ipynb/t1-start-pj-1.png)
* REKOGNITION_PROJECT_VERSION_ARN へ Amazon Resoure Name(ARN) の内容を設定し実行
    * <Amazon Resoure Name(ARN) の内容>の内容を今作成したARNへ置き換えて下さい
        * 例）　REKOGNITION_PROJECT_VERSION_ARN='arn:aws:rekognition:ap-northrast-1:XXXXXXX:project/demo-cats-project1/XXXXXXXXXXXX'


In [ ]:
# REKOGNITION_PROJECT_VERSION_ARN へ Amazon Resoure Name(ARN) の内容を設定
REKOGNITION_PROJECT_VERSION_ARN='<Amazon Resoure Name(ARN) の内容>'

# Rekognition Project 開始
!aws rekognition start-project-version \
  --project-version-arn {REKOGNITION_PROJECT_VERSION_ARN} \
  --min-inference-units 1 \
  --region ap-northeast-1

## 8. modelを利用した画像判定

In [ ]:
TEST_IMAGE_PATH='training1/mike.jpeg'
!aws s3 cp ./test_raw_images/mike.jpeg s3://{S3_BUCKET}/{TEST_IMAGE_PATH}

In [ ]:
import boto3, json

MIN_CONFIDENCE = 0
session = boto3.session.Session(region_name="ap-northeast-1")
client = session.client("rekognition")

from botocore.exceptions import ClientError

try:
    #Call DetectCustomLabels 
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': S3_BUCKET, 'Name': TEST_IMAGE_PATH}},
    MinConfidence=MIN_CONFIDENCE,
    ProjectVersionArn=REKOGNITION_PROJECT_VERSION_ARN)
    custom_labels_dict = response['CustomLabels']
    for label in custom_labels_dict :
        print(f'name: {label["Name"]}  confidence: {label["Confidence"]}')
        
except ClientError as err:
    print("Exception in get_custom_labels()")
    raise

## 9. modelの停止

In [ ]:
!aws rekognition stop-project-version \
  --project-version-arn {REKOGNITION_PROJECT_VERSION_ARN} \
  --region ap-northeast-1